In [3]:
%cd /root/YOLO-ViT-PGC/LA-Transformer/LATransformer
from __future__ import print_function

import os
import time
import glob
import random
import zipfile
from itertools import chain


os.environ['HF_ENDPOINT'] = "https://hf-mirror.com"
import timm
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from collections import OrderedDict
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.nn import init
import torch.optim as optim
from torchvision import models
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset

from LATransformer.model import ClassBlock, LATransformer, LATransformerTest
from LATransformer.utils import save_network, update_summary, get_id
from LATransformer.metrics import rank1, rank5, rank10, calc_map



device = "cuda"

/root/YOLO-ViT-PGC/LA-Transformer/LATransformer


In [4]:
batch_size = 8
gamma = 0.7
seed = 42

In [ ]:
# Load ViT
vit_base = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=751)
vit_base= vit_base.to(device)

# Create La-Transformer
model = LATransformerTest(vit_base, lmbd=8).to(device)

# Load LA-Transformer
name = "la_with_lmbd_8"
save_path = '/kaggle/input/lat-pri/pytorch/default/1/net_best.pth'
model.load_state_dict(torch.load(save_path), strict=False)
model.eval()